<h1> Import Libraries </h1>

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

<h1> Load DIM Table and Create Fact </h1>

In [34]:
# Load the mock dimension table 
df_dim = pd.read_csv(r'~\Python\MockIncidentDIM.csv')

# Create fake officer IDs
officer_ids = [f'Officer{i}' for i in range(1, 23)]

fact_data = []

# Iterate through unique incident numbers
unique_incident_numbers = df_dim['incident_number'].unique()

for incident_number in unique_incident_numbers:
    problem_type = df_dim.loc[df_dim['incident_number'] == incident_number, 'problem'].values[0]

    if problem_type in ["Extra Patrol", "Directed Patrol"]:
        num_assigned_officers = 1
    elif problem_type == "Traffic Stop":
        num_assigned_officers = np.random.randint(1, 3)
    else:
        num_assigned_officers = np.random.randint(1, 4)

    sampled_officers = np.random.choice(officer_ids, size=num_assigned_officers, replace=False)

    # Generate date times based on time_call_entered_queue in the dim table
    time_call_entered_queue = df_dim.loc[df_dim['incident_number'] == incident_number, 'time_call_entered_queue'].values[0]
    assigned_difference_secs = np.random.randint(10, 91)
    cleared_difference_secs = np.random.randint(120, 3600)
    assigned_datetime = pd.to_datetime(time_call_entered_queue) + timedelta(seconds=assigned_difference_secs)
    # Generate the arrived datetime based on urgency and assigned time
    priority_desc = df_dim.loc[df_dim['incident_number'] == incident_number, 'priority_desc'].values[0]
    if priority_desc in ["Emergency", "Urgent"]:
        time_difference_arrived = np.random.randint(10, 240)
    else:
        time_difference_arrived = np.random.randint(30, 1441)
    arrived_datetime = assigned_datetime + timedelta(seconds=time_difference_arrived)
    cleared_datetime = arrived_datetime + timedelta(seconds=cleared_difference_secs)
    
    response_secs = (arrived_datetime - assigned_datetime).total_seconds()
    total_secs = (cleared_datetime - assigned_datetime).total_seconds()
    time_on_call = (cleared_datetime - assigned_datetime).total_seconds()
    
    primary_flag = [1] + [0] * (num_assigned_officers - 1)
    
    for officer_id, primary_flag in zip(sampled_officers, primary_flag):
        fact_data.append([
            incident_number,
            officer_id,
            assigned_datetime,
            arrived_datetime,
            cleared_datetime,
            np.nan,
            response_secs,
            total_secs,
            time_on_call,
            primary_flag])

# Create the fact table DataFrame
fact_columns = ['incident_number', 'officer_id', 'assigned', 'arrived', 'cleared', 'delay', 'response_secs', 'total_secs', 'time_on_call', 'primary_flag']
df_fact = pd.DataFrame(fact_data, columns=fact_columns).reset_index().rename(columns={'index': 'id'})

print("Process Complete")

Process Complete


In [1]:
df_fact.head(5)

NameError: name 'df_fact' is not defined

In [36]:
df_fact.to_csv('MockIncidentFact.csv')